Maybe see also:
    - https://devopscube.com/run-docker-in-docker/
    - https://blog.skbali.com/2019/05/mongodb-replica-set-using-docker-compose/
    - https://gist.github.com/psychemedia/67a1c27ae1b0f0cee7ef

In [1]:
#%pip install pymongo
#%pip install --upgrade https://github.com/docker/docker-py/archive/master.zip

In [1]:
import docker

#Connect to docker
client = docker.from_env()

In [2]:
#Some helper functions via https://github.com/docker/docker-py

#Equivalent of docker ps
def docker_ps(c):
    return c.containers(quiet=False, all=False, trunc=True, latest=False, since=None, before=None, limit=-1)

In [3]:
client.containers.list()

[<Container: 9d2d38fd4d>, <Container: bc2fc497a9>, <Container: c2e7b1092b>]

In [4]:
container_ids = [c.id for c in client.containers.list()]
container_ids

['9d2d38fd4d7816aa0806bd9453525db89992db4bb38c576c9972c0bdebb4be1b',
 'bc2fc497a9f03af1296bb72f94e243fe816984098627e8679e5bbd24193bb3ed',
 'c2e7b1092bbf4b096afb871eb081d8ffbc2fbe97734d59a3970dab3ab962820f']

In [5]:
container = container_ids[0]

client.containers.get(container).attrs

{'Id': '9d2d38fd4d7816aa0806bd9453525db89992db4bb38c576c9972c0bdebb4be1b',
 'Created': '2020-11-04T16:08:16.1045496Z',
 'Path': '/usr/bin/mongod',
 'Args': ['--bind_ip_all', '--replSet', 'devrs'],
 'State': {'Status': 'running',
  'Running': True,
  'Paused': False,
  'Restarting': False,
  'OOMKilled': False,
  'Dead': False,
  'Pid': 3966,
  'ExitCode': 0,
  'Error': '',
  'StartedAt': '2020-11-04T16:08:16.962193Z',
  'FinishedAt': '0001-01-01T00:00:00Z'},
 'Image': 'sha256:525bd2016729a161cffa87b2ab606afabc5d95e744077259613d77b22533a3de',
 'ResolvConfPath': '/var/lib/docker/containers/9d2d38fd4d7816aa0806bd9453525db89992db4bb38c576c9972c0bdebb4be1b/resolv.conf',
 'HostnamePath': '/var/lib/docker/containers/9d2d38fd4d7816aa0806bd9453525db89992db4bb38c576c9972c0bdebb4be1b/hostname',
 'HostsPath': '/var/lib/docker/containers/9d2d38fd4d7816aa0806bd9453525db89992db4bb38c576c9972c0bdebb4be1b/hosts',
 'LogPath': '/var/lib/docker/containers/9d2d38fd4d7816aa0806bd9453525db89992db4bb38c576c99

In [6]:
#Find the local port bound for 27017/tcp for each server in the replica set
def get27017tcp_port(c, container):
    #cConfig = c.inspect_container(container)
    cConfig = client.containers.get(container).attrs
    return int(cConfig['NetworkSettings']['Ports']['27017/tcp'][0]['HostPort'])

def get27017tcp_ports(c, containers):
    ports={}
    for _container in containers:
        ports[_container.id]= get27017tcp_port(c, _container.id)
    return ports

In [7]:
get27017tcp_port(client, container)

30002

In [8]:
containers = client.containers.list()
get27017tcp_ports(client, containers)

{'9d2d38fd4d7816aa0806bd9453525db89992db4bb38c576c9972c0bdebb4be1b': 30002,
 'bc2fc497a9f03af1296bb72f94e243fe816984098627e8679e5bbd24193bb3ed': 30001,
 'c2e7b1092bbf4b096afb871eb081d8ffbc2fbe97734d59a3970dab3ab962820f': 30003}

In [9]:
def getContainIPaddress(c, container, network='jovyan_mongo-dev-net'):
    cConfig = client.containers.get(container).attrs
    return cConfig['NetworkSettings']['Networks'][network]['IPAddress']

def getContainIPaddresses(c, containers):
    ipaddresses={}
    for _container in containers:
        ipaddresses[_container]= getContainIPaddress(c, _container.id)
    return ipaddresses

In [10]:
getContainIPaddress(client, container)

'172.19.0.3'

In [11]:
getContainIPaddresses(client, containers)

{<Container: 9d2d38fd4d>: '172.19.0.3',
 <Container: bc2fc497a9>: '172.19.0.4',
 <Container: c2e7b1092b>: '172.19.0.2'}

In [12]:
#Initialise the replica set
from pymongo import MongoClient

#We'll use the 0th server in the set as a root node
mc = MongoClient('localhost', get27017tcp_port(client, container))

To initialise the replica set with a particular configuration, we need to define it using an object that has the following form:

```
rs_conf = {"_id" : "devrs",
        "members" : [{"_id" : 1, "host" : "172.19.0.2:30001"},
                     {"_id" : 2, "host" : "172.19.0.3:27017"},
                     {"_id" : 3, "host" : "172.19.0.4:27017"} ]}
```

In [58]:
container2 = container_ids[1]
mc2 = MongoClient('localhost', get27017tcp_port(client, container2))

In [13]:
rs_conf = {"_id" : "devrs",
        "members" : [{"_id" : 1, "host" : "mongo1"},
                     {"_id" : 2, "host" : "mongo2"},
                     {"_id" : 3, "host" : "mongo3"} ]}


In [14]:
#In the mongo console, we would typically use the command rs.config() to initial the replica set
#Here, we use the replSetInitiate admin command, applying it with the desired configuration
mc.admin.command( "replSetInitiate", rs_conf);
#mc.admin.command( "replSetInitiate") # use a default

OperationFailure: already initialized, full error: {'operationTime': Timestamp(1604506110, 1), 'ok': 0.0, 'errmsg': 'already initialized', 'code': 23, 'codeName': 'AlreadyInitialized', '$clusterTime': {'clusterTime': Timestamp(1604506110, 1), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}}

Alternatively, if we log in to one of continers eg for `mongo1` (`docker exec -it mongo1 /bin/bash`) and run `mongo` to access the command line, we can then run eg `rs.add('mongo2')`.

According to https://stackoverflow.com/a/54758273/454773 we can also do things like the following instead of `rd.add()`:

```python
import pymongo

conn = pymongo.MongoClient()
conf = conn.admin.command({'replSetGetConfig': 1})

conf['config']['members'].append({
    '_id': 3,
    'host': 'localhost:27020',
    'hidden': True,
    'priority': 0})
conf['config']['version'] += 1  # Bump the config version

res = conn.admin.command({'replSetReconfig': conf['config']})
print(res)
```

In [15]:
#We may need to wait a minute or two for the configuration to come up
#If you get an error message that suggests the configuration is up yet, wait a few seconds then rerun the cell
mc.admin.command('replSetGetStatus')

{'set': 'devrs',
 'date': datetime.datetime(2020, 11, 4, 16, 8, 32, 393000),
 'myState': 1,
 'term': 7,
 'syncingTo': '',
 'syncSourceHost': '',
 'syncSourceId': -1,
 'heartbeatIntervalMillis': 2000,
 'optimes': {'lastCommittedOpTime': {'ts': Timestamp(1604506110, 1), 't': 7},
  'readConcernMajorityOpTime': {'ts': Timestamp(1604506110, 1), 't': 7},
  'appliedOpTime': {'ts': Timestamp(1604506110, 1), 't': 7},
  'durableOpTime': {'ts': Timestamp(1604506110, 1), 't': 7}},
 'lastStableCheckpointTimestamp': Timestamp(1604506071, 1),
 'members': [{'_id': 0,
   'name': 'mongo2:27017',
   'health': 1.0,
   'state': 1,
   'stateStr': 'PRIMARY',
   'uptime': 16,
   'optime': {'ts': Timestamp(1604506110, 1), 't': 7},
   'optimeDate': datetime.datetime(2020, 11, 4, 16, 8, 30),
   'syncingTo': '',
   'syncSourceHost': '',
   'syncSourceId': -1,
   'infoMessage': 'could not find member to sync from',
   'electionTime': Timestamp(1604506098, 1),
   'electionDate': datetime.datetime(2020, 11, 4, 16, 8

In [17]:
testclient = MongoClient('localhost', get27017tcp_port(client, container))
testdb=testclient.testdb
testcollection=testdb.testcollection

In [47]:
#https://stackoverflow.com/questions/42632816/pymongo-set-log-level
from bson import SON

mc.admin.command(SON([
    ("setParameter", 1),
    ("logComponentVerbosity", {
        "storage": {
            "verbosity": 5,
            "journal": {
                "verbosity": 1
            }
        }
    })]))

{'was': {'verbosity': 0,
  'accessControl': {'verbosity': -1},
  'command': {'verbosity': -1},
  'control': {'verbosity': -1},
  'executor': {'verbosity': -1},
  'geo': {'verbosity': -1},
  'index': {'verbosity': -1},
  'network': {'verbosity': -1,
   'asio': {'verbosity': -1},
   'bridge': {'verbosity': -1}},
  'query': {'verbosity': -1},
  'replication': {'verbosity': -1,
   'heartbeats': {'verbosity': -1},
   'rollback': {'verbosity': -1}},
  'sharding': {'verbosity': -1, 'shardingCatalogRefresh': {'verbosity': -1}},
  'storage': {'verbosity': -1,
   'recovery': {'verbosity': -1},
   'journal': {'verbosity': -1}},
  'write': {'verbosity': -1},
  'ftdc': {'verbosity': -1},
  'tracking': {'verbosity': -1},
  'transaction': {'verbosity': -1}},
 'ok': 1.0,
 'operationTime': Timestamp(1604507020, 1),
 '$clusterTime': {'clusterTime': Timestamp(1604507020, 1),
  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'keyId': 0}}}

In [18]:
testcollection.insert_one({'name':'test1'})

In [19]:
for x in range(0,10):
    testcollection.insert_one({'name':'test'+str(x)})

In [20]:
for ff in testcollection.find():
    print(ff)

{'_id': ObjectId('5fa2c9266dd0e7f5ad614b66'), 'name': 'test1'}
{'_id': ObjectId('5fa2c94f6dd0e7f5ad614b67'), 'name': 'test0'}
{'_id': ObjectId('5fa2c94f6dd0e7f5ad614b68'), 'name': 'test1'}
{'_id': ObjectId('5fa2c94f6dd0e7f5ad614b69'), 'name': 'test2'}
{'_id': ObjectId('5fa2c9506dd0e7f5ad614b6a'), 'name': 'test3'}
{'_id': ObjectId('5fa2c9506dd0e7f5ad614b6b'), 'name': 'test4'}
{'_id': ObjectId('5fa2c9506dd0e7f5ad614b6c'), 'name': 'test5'}
{'_id': ObjectId('5fa2c9506dd0e7f5ad614b6d'), 'name': 'test6'}
{'_id': ObjectId('5fa2c9506dd0e7f5ad614b6e'), 'name': 'test7'}
{'_id': ObjectId('5fa2c9506dd0e7f5ad614b6f'), 'name': 'test8'}
{'_id': ObjectId('5fa2c9506dd0e7f5ad614b70'), 'name': 'test9'}
{'_id': ObjectId('5fa2c9546dd0e7f5ad614b71'), 'name': 'test0'}
{'_id': ObjectId('5fa2c9546dd0e7f5ad614b72'), 'name': 'test1'}
{'_id': ObjectId('5fa2c9546dd0e7f5ad614b73'), 'name': 'test2'}
{'_id': ObjectId('5fa2c9546dd0e7f5ad614b74'), 'name': 'test3'}
{'_id': ObjectId('5fa2c9546dd0e7f5ad614b75'), 'name': '

We can model network failures by setting up firewall rules to block messages being passed between particular containers.

In [36]:
##https://github.com/dcm-oss/blockade/

#
#  Copyright (C) 2014 Dell, Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

import random
import string
import subprocess

import collections.abc


#---errors.py
class BlockadeError(Exception):
    """Expected error within Blockade
    """


class BlockadeConfigError(BlockadeError):
    """Error in configuration
    """


class AlreadyInitializedError(BlockadeError):
    """Blockade already created in this context
    """


class NotInitializedError(BlockadeError):
    """Blockade not created in this context
    """


class InconsistentStateError(BlockadeError):
    """Blockade state is inconsistent (partially created or destroyed)
    """
    
#---



def parse_partition_index(blockade_id, chain):
    prefix = "%s-p" % (blockade_id,)
    if chain and chain.startswith(prefix):
        try:
            return int(chain[len(prefix):])
        except ValueError:
            pass
    raise ValueError("chain %s is not a blockade partition" % (chain,))


def partition_chain_name(blockade_id, partition_index):
    return "%s-p%s" % (blockade_id, partition_index)


def iptables_call_output(*args):
    cmd = ["iptables", "-n"] + list(args)
    try:
        output = subprocess.check_output(cmd)
        return output.decode().split("\n")
    except subprocess.CalledProcessError:
        raise BlockadeError("Problem calling '%s'" % " ".join(cmd))


def iptables_call(*args):
    cmd = ["iptables"] + list(args)
    try:
        subprocess.check_call(cmd)
    except subprocess.CalledProcessError:
        raise BlockadeError("Problem calling '%s'" % " ".join(cmd))


def iptables_get_chain_rules(chain):
    if not chain:
        raise ValueError("invalid chain")
    lines = iptables_call_output("-L", chain)
    if len(lines) < 2:
        raise BlockadeError("Can't understand iptables output: \n%s" %
                            "\n".join(lines))

    chain_line, header_line = lines[:2]
    if not (chain_line.startswith("Chain " + chain) and
            header_line.startswith("target")):
        raise BlockadeError("Can't understand iptables output: \n%s" %
                            "\n".join(lines))
    return lines[2:]


def iptables_get_source_chains(blockade_id):
    """Get a map of blockade chains IDs -> list of IPs targeted at them

    For figuring out which container is in which partition
    """
    result = {}
    if not blockade_id:
        raise ValueError("invalid blockade_id")
    lines = iptables_get_chain_rules("FORWARD")

    for line in lines:
        parts = line.split()
        if len(parts) < 4:
            continue
        try:
            partition_index = parse_partition_index(blockade_id, parts[0])
        except ValueError:
            continue  # not a rule targetting a blockade chain

        source = parts[3]
        if source:
            result[source] = partition_index
    return result


def iptables_delete_rules(chain, predicate):
    if not chain:
        raise ValueError("invalid chain")
    if not isinstance(predicate, collections.Callable):
        raise ValueError("invalid predicate")

    lines = iptables_get_chain_rules(chain)

    # TODO this is susceptible to check-then-act races.
    # better to ultimately switch to python-iptables if it becomes less buggy
    for index, line in reversed(list(enumerate(lines, 1))):
        line = line.strip()
        if line and predicate(line):
            iptables_call("-D", chain, str(index))


def iptables_delete_blockade_rules(blockade_id):
    def predicate(rule):
        target = rule.split()[0]
        try:
            parse_partition_index(blockade_id, target)
        except ValueError:
            return False
        return True
    iptables_delete_rules("FORWARD", predicate)


def iptables_delete_blockade_chains(blockade_id):
    if not blockade_id:
        raise ValueError("invalid blockade_id")

    lines = iptables_call_output("-L")
    for line in lines:
        parts = line.split()
        if len(parts) >= 2 and parts[0] == "Chain":
            chain = parts[1]
            try:
                parse_partition_index(blockade_id, chain)
            except ValueError:
                continue
            # if we are a valid blockade chain, flush and delete
            iptables_call("-F", chain)
            iptables_call("-X", chain)


def iptables_insert_rule(chain, src=None, dest=None, target=None):
    """Insert a new rule in the chain
    """
    if not chain:
        raise ValueError("Invalid chain")
    if not target:
        raise ValueError("Invalid target")
    if not (src or dest):
        raise ValueError("Need src, dest, or both")

    args = ["-I", chain]
    if src:
        args += ["-s", src]
    if dest:
        args += ["-d", dest]
    args += ["-j", target]
    iptables_call(*args)


def iptables_create_chain(chain):
    """Create a new chain
    """
    if not chain:
        raise ValueError("Invalid chain")
    iptables_call("-N", chain)


def clear_iptables(blockade_id):
    """Remove all iptables rules and chains related to this blockade
    """
    # first remove refererences to our custom chains
    iptables_delete_blockade_rules(blockade_id)

    # then remove the chains themselves
    iptables_delete_blockade_chains(blockade_id)


def partition_containers(blockade_id, partitions):
    if not partitions or len(partitions) == 1:
        return
    for index, partition in enumerate(partitions, 1):
        chain_name = partition_chain_name(blockade_id, index)

        # create chain for partition and block traffic TO any other partition
        iptables_create_chain(chain_name)
        for other in partitions:
            if partition is other:
                continue
            for container in other:
                if container.ip_address:
                    iptables_insert_rule(chain_name, dest=container.ip_address,
                                         target="DROP")

        # direct traffic FROM any container in the partition to the new chain
        for container in partition:
            iptables_insert_rule("FORWARD", src=container.ip_address,
                                 target=chain_name)

In [37]:
class netobj():
    def __init__(self, ip_address):
        self.ip_address = ip_address

We assign the IP addresses of the containers into different partitions.

In [56]:
#In this case, let's be cruel and put the primary in a partition on its own
partition_containers('test1w2s5', [ [netobj('172.19.0.2')],[netobj('172.19.0.3'),netobj('172.19.0.4')]])
#Wait a bit before generating the log...

In [57]:
mc.admin.command('replSetGetStatus')

{'set': 'devrs',
 'date': datetime.datetime(2020, 11, 4, 16, 30, 5, 970000),
 'myState': 1,
 'term': 7,
 'syncingTo': '',
 'syncSourceHost': '',
 'syncSourceId': -1,
 'heartbeatIntervalMillis': 2000,
 'optimes': {'lastCommittedOpTime': {'ts': Timestamp(1604507400, 1), 't': 7},
  'readConcernMajorityOpTime': {'ts': Timestamp(1604507400, 1), 't': 7},
  'appliedOpTime': {'ts': Timestamp(1604507400, 1), 't': 7},
  'durableOpTime': {'ts': Timestamp(1604507400, 1), 't': 7}},
 'lastStableCheckpointTimestamp': Timestamp(1604507350, 1),
 'members': [{'_id': 0,
   'name': 'mongo2:27017',
   'health': 1.0,
   'state': 1,
   'stateStr': 'PRIMARY',
   'uptime': 1309,
   'optime': {'ts': Timestamp(1604507400, 1), 't': 7},
   'optimeDate': datetime.datetime(2020, 11, 4, 16, 30),
   'syncingTo': '',
   'syncSourceHost': '',
   'syncSourceId': -1,
   'infoMessage': '',
   'electionTime': Timestamp(1604506098, 1),
   'electionDate': datetime.datetime(2020, 11, 4, 16, 8, 18),
   'configVersion': 1,
   's

In [60]:
mc.admin.command('printSlaveReplicationInfo')

OperationFailure: no such command: 'printSlaveReplicationInfo', full error: {'operationTime': Timestamp(1604507560, 1), 'ok': 0.0, 'errmsg': "no such command: 'printSlaveReplicationInfo'", 'code': 59, 'codeName': 'CommandNotFound', '$clusterTime': {'clusterTime': Timestamp(1604507560, 1), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}}

In [49]:
!docker ps

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                      NAMES
9d2d38fd4d78        mongo:4.0.4         "/usr/bin/mongod --b…"   16 minutes ago      Up 16 minutes       0.0.0.0:30002->27017/tcp   mongo2
bc2fc497a9f0        mongo:4.0.4         "/usr/bin/mongod --b…"   16 minutes ago      Up 16 minutes       0.0.0.0:30001->27017/tcp   mongo1
c2e7b1092bbf        mongo:4.0.4         "/usr/bin/mongod --b…"   16 minutes ago      Up 16 minutes       0.0.0.0:30003->27017/tcp   mongo3


In [50]:
!docker logs mongo1 > mongo1.txt
!docker logs mongo2 > mongo2.txt
!docker logs mongo3 > mongo3.txt

In [51]:
!tail -n 30 mongo1.txt

2020-11-04T16:08:17.034+0000 I CONTROL  [initandlisten]     target_arch: x86_64
2020-11-04T16:08:17.034+0000 I CONTROL  [initandlisten] options: { net: { bindIpAll: true }, replication: { replSet: "devrs" } }
2020-11-04T16:08:17.035+0000 W STORAGE  [initandlisten] Detected unclean shutdown - /data/db/mongod.lock is not empty.
2020-11-04T16:08:17.036+0000 I STORAGE  [initandlisten] Detected data files in /data/db created by the 'wiredTiger' storage engine, so setting the active storage engine to 'wiredTiger'.
2020-11-04T16:08:17.037+0000 W STORAGE  [initandlisten] Recovering data from the last clean checkpoint.
2020-11-04T16:08:17.037+0000 I STORAGE  [initandlisten] wiredtiger_open config: create,cache_size=984M,session_max=20000,eviction=(threads_min=4,threads_max=4),config_base=false,statistics=(fast),log=(enabled=true,archive=true,path=journal,compressor=snappy),file_manager=(close_idle_time=100000),statistics_log=(wait=0),verbose=(recovery_progress),
2020-11-04T16:08:17.886+0000 I S

In [53]:
!tail -n 80 mongo2.txt

2020-11-04T16:24:26.854+0000 D STORAGE  [WTJournalFlusher] flushed journal
2020-11-04T16:24:26.955+0000 D STORAGE  [WTJournalFlusher] flushed journal
2020-11-04T16:24:27.000+0000 D STORAGE  [ftdc] setting timestamp read source: 1, provided timestamp: none
2020-11-04T16:24:27.001+0000 D STORAGE  [ftdc] NamespaceUUIDCache: registered namespace local.oplog.rs with UUID 953dc1ca-6a48-4ca2-ada0-020dafcc68b0
2020-11-04T16:24:27.017+0000 D STORAGE  [rsSync-0] NamespaceUUIDCache: registered namespace local.replset.minvalid with UUID 0eb68e90-6a8d-4f73-9f9d-e796e743c141
2020-11-04T16:24:27.017+0000 D STORAGE  [rsSync-0] WT begin_transaction for snapshot id 4694
2020-11-04T16:24:27.017+0000 D STORAGE  [rsSync-0] WT rollback_transaction for snapshot id 4694
2020-11-04T16:24:27.055+0000 D STORAGE  [WTJournalFlusher] flushed journal
2020-11-04T16:24:27.156+0000 D STORAGE  [WTJournalFlusher] flushed journal
2020-11-04T16:24:27.257+0000 D STORAGE  [WTJournalFlusher] flushed journal
2020-11-04T16:24:2

In [54]:
!tail -n 30 mongo3.txt

2020-11-04T16:08:18.124+0000 I REPL     [replexec-0] New replica set config in use: { _id: "devrs", version: 1, protocolVersion: 1, writeConcernMajorityJournalDefault: true, members: [ { _id: 0, host: "mongo3:27017", arbiterOnly: false, buildIndexes: true, hidden: false, priority: 1.0, tags: {}, slaveDelay: 0, votes: 1 } ], settings: { chainingAllowed: true, heartbeatIntervalMillis: 2000, heartbeatTimeoutSecs: 10, electionTimeoutMillis: 10000, catchUpTimeoutMillis: -1, catchUpTakeoverDelayMillis: 30000, getLastErrorModes: {}, getLastErrorDefaults: { w: 1, wtimeout: 0 }, replicaSetId: ObjectId('5fa2cedecda5066af5fece76') } }
2020-11-04T16:08:18.125+0000 I REPL     [replexec-0] This node is mongo3:27017 in the config
2020-11-04T16:08:18.125+0000 I REPL     [replexec-0] transition to STARTUP2 from STARTUP
2020-11-04T16:08:18.125+0000 I REPL     [replexec-0] Starting replication storage threads
2020-11-04T16:08:18.126+0000 I NETWORK  [LogicalSessionCacheRefresh] Starting new replica set mo

In [38]:
clear_iptables('test1w2s3')

In [39]:
!docker logs mongo1 > mongo1.txt
!docker logs mongo2 > mongo2.txt
!docker logs mongo3 > mongo3.txt

In [40]:
!tail -n 30 mongo1.txt

2020-11-04T16:08:17.034+0000 I CONTROL  [initandlisten] modules: none
2020-11-04T16:08:17.034+0000 I CONTROL  [initandlisten] build environment:
2020-11-04T16:08:17.034+0000 I CONTROL  [initandlisten]     distmod: ubuntu1604
2020-11-04T16:08:17.034+0000 I CONTROL  [initandlisten]     distarch: x86_64
2020-11-04T16:08:17.034+0000 I CONTROL  [initandlisten]     target_arch: x86_64
2020-11-04T16:08:17.034+0000 I CONTROL  [initandlisten] options: { net: { bindIpAll: true }, replication: { replSet: "devrs" } }
2020-11-04T16:08:17.035+0000 W STORAGE  [initandlisten] Detected unclean shutdown - /data/db/mongod.lock is not empty.
2020-11-04T16:08:17.036+0000 I STORAGE  [initandlisten] Detected data files in /data/db created by the 'wiredTiger' storage engine, so setting the active storage engine to 'wiredTiger'.
2020-11-04T16:08:17.037+0000 W STORAGE  [initandlisten] Recovering data from the last clean checkpoint.
2020-11-04T16:08:17.037+0000 I STORAGE  [initandlisten] wiredtiger_open config: 

In [41]:
!tail -n 30 mongo2.txt

2020-11-04T16:08:18.245+0000 I REPL     [replexec-0] New replica set config in use: { _id: "devrs", version: 1, protocolVersion: 1, writeConcernMajorityJournalDefault: true, members: [ { _id: 0, host: "mongo2:27017", arbiterOnly: false, buildIndexes: true, hidden: false, priority: 1.0, tags: {}, slaveDelay: 0, votes: 1 } ], settings: { chainingAllowed: true, heartbeatIntervalMillis: 2000, heartbeatTimeoutSecs: 10, electionTimeoutMillis: 10000, catchUpTimeoutMillis: -1, catchUpTakeoverDelayMillis: 30000, getLastErrorModes: {}, getLastErrorDefaults: { w: 1, wtimeout: 0 }, replicaSetId: ObjectId('5fa2c6bb5354956582583364') } }
2020-11-04T16:08:18.245+0000 I REPL     [replexec-0] This node is mongo2:27017 in the config
2020-11-04T16:08:18.245+0000 I REPL     [replexec-0] transition to STARTUP2 from STARTUP
2020-11-04T16:08:18.245+0000 I REPL     [replexec-0] Starting replication storage threads
2020-11-04T16:08:18.247+0000 I REPL     [replexec-0] transition to RECOVERING from STARTUP2
2020

In [42]:
!tail -n 30 mongo3.txt

2020-11-04T16:08:18.124+0000 I REPL     [replexec-0] New replica set config in use: { _id: "devrs", version: 1, protocolVersion: 1, writeConcernMajorityJournalDefault: true, members: [ { _id: 0, host: "mongo3:27017", arbiterOnly: false, buildIndexes: true, hidden: false, priority: 1.0, tags: {}, slaveDelay: 0, votes: 1 } ], settings: { chainingAllowed: true, heartbeatIntervalMillis: 2000, heartbeatTimeoutSecs: 10, electionTimeoutMillis: 10000, catchUpTimeoutMillis: -1, catchUpTakeoverDelayMillis: 30000, getLastErrorModes: {}, getLastErrorDefaults: { w: 1, wtimeout: 0 }, replicaSetId: ObjectId('5fa2cedecda5066af5fece76') } }
2020-11-04T16:08:18.125+0000 I REPL     [replexec-0] This node is mongo3:27017 in the config
2020-11-04T16:08:18.125+0000 I REPL     [replexec-0] transition to STARTUP2 from STARTUP
2020-11-04T16:08:18.125+0000 I REPL     [replexec-0] Starting replication storage threads
2020-11-04T16:08:18.126+0000 I NETWORK  [LogicalSessionCacheRefresh] Starting new replica set mo